# Keyword graph Generation

> moved to make_keyword_graph.py

In [ ]:
!pip show openai

In [3]:
!pip freeze > ipynb_requirements.txt

## 법률 용어 정의가 포함된 문장 추출

정의문장만 포함할 경우 항과의 관계를 추출하지 못함

In [ ]:
import GraphDB.utils as utils

same_sentences, include_sentences = utils.extract_definition_sentences("C:/Users/Shic/legal_graph/results/1-2/DCM_1-2_law_main_clause.json")

In [ ]:
print(f"동의어 관계 : {len(same_sentences)}\nexample : {same_sentences[0]}\n\n포함 관계 : {len(include_sentences)}\nexample : {include_sentences[0]}")

In [ ]:
same_sentences[0]

In [24]:
# 그냥 리스트 순회하면서 넣는게 나을듯
import copy

definition_sentences = copy.deepcopy(same_sentences)

for sentence in include_sentences:
    if sentence not in definition_sentences:
        definition_sentences.append(sentence)
    else:
        continue

In [ ]:
len(same_sentences), len(include_sentences), len(definition_sentences)

## make jsonl file

In [ ]:
# make triplet
utils.make_triplet_jsonl(definition_sentences,
                         data_type="법",
                         prompt_path="C:/Users/Shic/legal_graph/codes/GraphDB/prompt/triplet_extractor.txt",
                         jsonl_path="C:/Users/Shic/legal_graph/results/1-2/jsonl/definition_text.jsonl",
                         meta_path="C:/Users/Shic/legal_graph/results/1-2/jsonl/definition_metadata.csv")

In [27]:
# 반횐되는 batch_id 활용

batch_result = utils.make_triplet_batch("C:/Users/Shic/legal_graph/results/1-2/jsonl/definition_text.jsonl")
batch_id = batch_result.id
# batch_id = 'batch_670cb1e427248190a458b56b75d21a0f'

In [ ]:
import GraphDB.utils as utils

status = utils.get_batch_status(batch_id)
status.status

In [ ]:
output_file_id = status.output_file_id
output_file_id

In [ ]:
utils.get_batch_answer(output_file_id=output_file_id,
                       jsonl_path="C:/Users/Shic/legal_graph/results/1-2/jsonl/triplet_output.jsonl")

In [ ]:
import GraphDB.utils as utils

triplets = utils.get_triplet(result_path="C:/Users/Shic/legal_graph/results/1-2/jsonl/triplet_output.jsonl", 
                             meta_path="C:/Users/Shic/legal_graph/results/1-2/jsonl/definition_metadata.csv")
triplets[0]

In [ ]:
for idx, triplet in enumerate(triplets):
    print(f"{idx} : {triplet[1]}")

In [45]:
include = [x[1] for x in triplets if x[1] == "INCLUDED_IN"]
same = [x[1] for x in triplets if x[1] == "SAME_AS"]

In [ ]:
len(include), len(same)

## keyword graph 구축

In [ ]:
import json
from GraphDB.LegalGraphDB import LegalGraphDB

CONFIG_PATH = "C:/Users/Shic/legal_graph/codes/configs/config.json"

# config_file 가져오기
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)
    print(f"Loaded config data from {CONFIG_PATH}")

dbms = LegalGraphDB(auradb=False, config=config)

In [ ]:
# 이전 graph 삭제
dbms.delete_all_relationship("legal-graph")
dbms.delete_all_nodes("legal-graph")

In [ ]:
triplets[0]

In [ ]:
# keyword graph 구축
dbms.create_keyword_graph(triplets, "legal-graph")

## Keyword graph 후처리

- 프롬프트 단에서 해결이 안될 경우
- jsonl에서 직접 보고, 정규표현식 등을 이용해서 컷하기 -> (*.등)
- make_keyword_graph 단에 모듈로 집어넣기


### law_main

In [1]:
import GraphDB.utils as utils

triplets = utils.get_triplet(result_path="../results/1-2/jsonl/01_law_main_triplet_preprocessed_chunking_cot.jsonl", 
                             meta_path="../results/1-2/jsonl/01_law_main_definition_metadata_preprocessed_chunking_cot.csv")
triplets

## 363 개의 데이터 로드됨. 데이터 예시 : {'id': 'batch_req_6715b0bad7688190a086e5cb20cc7f2f', 'custom_id': 'request-0', 'response': {'status_code': 200, 'request_id': '934df407b2e2a40057c6149a0464be5d', 'body': {'id': 'chatcmpl-AKbduznkNFP6nSdCNRhoBVe1RYziF', 'object': 'chat.completion', 'created': 1729474622, 'model': 'gpt-4o-2024-08-06', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '(금전, 그 밖의 재산적 가치가 있는 것 | SAME_AS | 금전등)  \n(위험 | SAME_AS | 투자성)  ', 'refusal': None}, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 1533, 'completion_tokens': 34, 'total_tokens': 1567, 'prompt_tokens_details': {'cached_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'system_fingerprint': 'fp_45c6de4934'}}, 'error': None}


100%|██████████| 363/363 [00:00<00:00, 2377.99it/s]

## edge :  {'type': 'SAME_AS', 'reference': '법_제3조제1항_금융투자상품'}
## edge :  {'type': 'SAME_AS', 'reference': '법_제3조제1항_금융투자상품'}
## edge :  {'type': 'INCLUDED_IN', 'reference': '법_제3조제1항_금융투자상품'}
## edge :  {'type': 'INCLUDED_IN', 'reference': '법_제3조제1항_금융투자상품'}
output format error: not enough values to unpack (expected 3, got 1)

output :output:
## edge :  {'type': 'INCLUDED_IN', 'reference': '법_제6조제7항_금융투자업'}
## edge :  {'type': 'INCLUDED_IN', 'reference': '법_제6조제7항_금융투자업'}
## edge :  {'type': 'INCLUDED_IN', 'reference': '법_제6조제7항_금융투자업'}
output format error: not enough values to unpack (expected 3, got 1)

output :output:
## edge :  {'type': 'INCLUDED_IN', 'reference': '법_제6조제10항_금융투자업'}
output format error: not enough values to unpack (expected 3, got 1)

output :output:
## edge :  {'type': 'SAME_AS', 'reference': '법_제8조제9항_금융투자업자'}
## edge :  {'type': 'SAME_AS', 'reference': '법_제8조제9항_금융투자업자'}
## edge :  {'type': 'INCLUDED_IN', 'reference': '법_제8조의2제5항_금융투자상품시장 등'}
## edge :  {'type'

[({'law_index': '법_제3조제1항',
   'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )\n',
   'created_date': '시행 2024. 8. 14.',
   'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정\n',
   'source': '국가법령정보센터',
   'reference': '법_제1편 총칙_제3조제1항_금융투자상품',
   'keyword_name': '금전, 그 밖의 재산적 가치가 있는 것'},
  {'type': 'SAME_AS', 'reference': '법_제3조제1항_금융투자상품'},
  {'law_index': '법_제3조제1항',
   'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )\n',
   'created_date': '시행 2024. 8. 14.',
   'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정\n',
   'source': '국가법령정보센터',
   'reference': '법_제1편 총칙_제3조제1항_금융투자상품',
   'keyword_name': '금전등'}),
 ({'law_index': '법_제3조제1항',
   'document_title': '자본시장과 금융투자업에 관한 법률 ( 약칭: 자본시장법 )\n',
   'created_date': '시행 2024. 8. 14.',
   'revise_info': '법률 제20305호, 2024. 2. 13., 일부개정\n',
   'source': '국가법령정보센터',
   'reference': '법_제1편 총칙_제3조제1항_금융투자상품',
   'keyword_name': '위험'},
  {'type': 'SAME_AS', 'reference': '법_제3조제1항_금융투자상품'},
  {'law_index': '법_제3조제1항',
   'document_title': '자본시

In [2]:
len(triplets)

1164

In [3]:
for triplet in triplets:
    s, edge, o = triplet
    print(f"{edge['reference']} : {s['keyword_name']}-[{edge['type']}]-{o['keyword_name']}")


법_제3조제1항_금융투자상품 : 금전, 그 밖의 재산적 가치가 있는 것-[SAME_AS]-금전등
법_제3조제1항_금융투자상품 : 위험-[SAME_AS]-투자성
법_제3조제1항_금융투자상품 : 수익증권발행신탁이 아닌 신탁-[INCLUDED_IN]-관리형신탁
법_제3조제1항_금융투자상품 : 수익권-[INCLUDED_IN]-관리형신탁
법_제6조제7항_금융투자업 : 금융투자상품-[INCLUDED_IN]-금융투자상품등
법_제6조제7항_금융투자업 : 대통령령으로 정하는 투자대상자산-[INCLUDED_IN]-금융투자상품등
법_제6조제7항_금융투자업 : 금융투자상품등에 대한 투자판단-[INCLUDED_IN]-투자자문업
법_제6조제10항_금융투자업 : 일반 사모집합투자기구-[INCLUDED_IN]-일반 사모집합투자기구등
법_제8조제9항_금융투자업자 : 은행법 제2조의 은행-[SAME_AS]-은행
법_제8조제9항_금융투자업자 : 보험업법 제2조의 보험회사-[SAME_AS]-보험회사
법_제8조의2제5항_금융투자상품시장 등 : 매매체결대상상품-[INCLUDED_IN]-주권 및 그 밖에 대통령령으로 정하는 증권
법_제8조의2제5항_금융투자상품시장 등 : 다자간매매체결업무-[INCLUDED_IN]-매매 또는 그 중개ㆍ주선이나 대리 업무
법_제9조제10항_그 밖의 용어의 정의 : 상장법인-[SAME_AS]-증권시장에 상장된 증권을 발행한 법인
법_제9조제10항_그 밖의 용어의 정의 : 상장증권-[INCLUDED_IN]-증권시장에 상장된 증권
법_제9조제10항_그 밖의 용어의 정의 : 비상장법인-[SAME_AS]-상장법인을 제외한 법인
법_제9조제10항_그 밖의 용어의 정의 : 한국금융투자협회-[SAME_AS]-협회
법_제9조제10항_그 밖의 용어의 정의 : 한국예탁결제원-[SAME_AS]-예탁결제원
법_제9조제10항_그 밖의 용어의 정의 : 제323조의3에 따라 인가를 받은 자-[SAME_AS]-금융투자상품거래청산회사
법_제9조제10항_그 밖의 용어의 정의 : 제324조제1항에 따라 인

In [4]:
import GraphDB.process as process
remove_idx, new_triplets = process.postprocess(triplets)

In [5]:
print(f"original triplets : {len(triplets)}, new triplets : {len(new_triplets)}")

original triplets : 1164, new triplets : 1123


In [6]:
remove_idx.sort()

In [7]:
for idx in remove_idx:
    s, edge, o = triplets[idx]
    print(f"[{idx}] | [{edge['reference']}] | {s['keyword_name']}-[{edge['type']}]-{o['keyword_name']}")

[4] | [법_제6조제7항_금융투자업] | 금융투자상품-[INCLUDED_IN]-금융투자상품등
[7] | [법_제6조제10항_금융투자업] | 일반 사모집합투자기구-[INCLUDED_IN]-일반 사모집합투자기구등
[69] | [법_제45조제1항_정보교류의 차단] | 금융투자업-[INCLUDED_IN]-금융투자업등
[70] | [법_제45조제1항_정보교류의 차단] | 제40조제1항 각 호의 업무-[INCLUDED_IN]-금융투자업등
[73] | [법_제45조제1항_정보교류의 차단] | 미공개중요정보 등 대통령령으로 정하는 정보-[INCLUDED_IN]-제174조제1항 각 호 외의 부분에 따른 정보
[82] | [법_제63조제1항_임직원의 금융투자상품 매매] | 자기의 계산으로 대통령령으로 정하는 금융투자상품을 매매하는 경우-[INCLUDED_IN]-다음 각 호의 방법
[217] | [법_제162조제1항_거짓의 기재 등에 의한 배상책임] | 사업보고서-[INCLUDED_IN]-사업보고서등
[224] | [법_제165조의4제1항_합병 등의 특례] | 각 호의 어느 하나에 해당하는 행위-[SAME_AS]-합병 등
[239] | [법_제169조제3항_회계감사인에 의한 감사증명] | 외국법인-[INCLUDED_IN]-외국법인등
[243] | [법_제172조제1항_내부자의 단기매매차익 반환] | 「상법」 제401조의2제1항 각 호의 자-[INCLUDED_IN]-임원
[269] | [법_제176조제4항_시세조종행위 등의 금지] | 매매-[INCLUDED_IN]-매매등
[319] | [법_제230조제1항_환매금지형집합투자기구] | 집합투자업자-[INCLUDED_IN]-집합투자업자등
[330] | [법_제235조제2항_환매청구 및 방법 등] | 해산-[INCLUDED_IN]-해산등
[361] | [법_제249조의18제2항_상호출자제한기업집단 계열 기관전용 사모집합투자기구 등에 대한 제한] | 기관전용 사모집합투자기구등-[SAME_AS]-기관전용 사모집합투자기구
[381] | [

In [8]:
for idx, triplet in enumerate(new_triplets):
    s, edge, o = triplet
    print(f"[{idx}] | {edge['reference']} : {s['keyword_name']}-[{edge['type']}]-{o['keyword_name']}")

[0] | 법_제3조제1항_금융투자상품 : 금전, 그 밖의 재산적 가치가 있는 것-[SAME_AS]-금전등
[1] | 법_제3조제1항_금융투자상품 : 위험-[SAME_AS]-투자성
[2] | 법_제3조제1항_금융투자상품 : 수익증권발행신탁이 아닌 신탁-[INCLUDED_IN]-관리형신탁
[3] | 법_제3조제1항_금융투자상품 : 수익권-[INCLUDED_IN]-관리형신탁
[4] | 법_제6조제7항_금융투자업 : 대통령령으로 정하는 투자대상자산-[INCLUDED_IN]-금융투자상품등
[5] | 법_제6조제7항_금융투자업 : 금융투자상품등에 대한 투자판단-[INCLUDED_IN]-투자자문업
[6] | 법_제8조제9항_금융투자업자 : 은행법 제2조의 은행-[SAME_AS]-은행
[7] | 법_제8조제9항_금융투자업자 : 보험업법 제2조의 보험회사-[SAME_AS]-보험회사
[8] | 법_제8조의2제5항_금융투자상품시장 등 : 매매체결대상상품-[INCLUDED_IN]-주권 및 그 밖에 대통령령으로 정하는 증권
[9] | 법_제8조의2제5항_금융투자상품시장 등 : 다자간매매체결업무-[INCLUDED_IN]-매매 또는 그 중개ㆍ주선이나 대리 업무
[10] | 법_제9조제10항_그 밖의 용어의 정의 : 상장법인-[SAME_AS]-증권시장에 상장된 증권을 발행한 법인
[11] | 법_제9조제10항_그 밖의 용어의 정의 : 상장증권-[INCLUDED_IN]-증권시장에 상장된 증권
[12] | 법_제9조제10항_그 밖의 용어의 정의 : 비상장법인-[SAME_AS]-상장법인을 제외한 법인
[13] | 법_제9조제10항_그 밖의 용어의 정의 : 한국금융투자협회-[SAME_AS]-협회
[14] | 법_제9조제10항_그 밖의 용어의 정의 : 한국예탁결제원-[SAME_AS]-예탁결제원
[15] | 법_제9조제10항_그 밖의 용어의 정의 : 제323조의3에 따라 인가를 받은 자-[SAME_AS]-금융투자상품거래청산회사
[16] | 법_제9조제10항_그 밖의 용어의 정의 : 제324조제1항

In [9]:
# 마음에 안드는 triplet idx 뽑아서 넣기
manual_remove_idx = [751, 890, 1017, 1018, 1019, 1020, 1021, 1022, 1023, 1024, 1025, 1026]

remove_idx, new_triplets = process.postprocess(triplets, manual_remove_idx)
remove_idx.sort()

In [10]:
for idx in remove_idx:
    s, edge, o = triplets[idx]
    print(f"[{idx}] | [{edge['reference']}] | {s['keyword_name']}-[{edge['type']}]-{o['keyword_name']}")

[4] | [법_제6조제7항_금융투자업] | 금융투자상품-[INCLUDED_IN]-금융투자상품등
[7] | [법_제6조제10항_금융투자업] | 일반 사모집합투자기구-[INCLUDED_IN]-일반 사모집합투자기구등
[69] | [법_제45조제1항_정보교류의 차단] | 금융투자업-[INCLUDED_IN]-금융투자업등
[70] | [법_제45조제1항_정보교류의 차단] | 제40조제1항 각 호의 업무-[INCLUDED_IN]-금융투자업등
[73] | [법_제45조제1항_정보교류의 차단] | 미공개중요정보 등 대통령령으로 정하는 정보-[INCLUDED_IN]-제174조제1항 각 호 외의 부분에 따른 정보
[82] | [법_제63조제1항_임직원의 금융투자상품 매매] | 자기의 계산으로 대통령령으로 정하는 금융투자상품을 매매하는 경우-[INCLUDED_IN]-다음 각 호의 방법
[217] | [법_제162조제1항_거짓의 기재 등에 의한 배상책임] | 사업보고서-[INCLUDED_IN]-사업보고서등
[224] | [법_제165조의4제1항_합병 등의 특례] | 각 호의 어느 하나에 해당하는 행위-[SAME_AS]-합병 등
[239] | [법_제169조제3항_회계감사인에 의한 감사증명] | 외국법인-[INCLUDED_IN]-외국법인등
[243] | [법_제172조제1항_내부자의 단기매매차익 반환] | 「상법」 제401조의2제1항 각 호의 자-[INCLUDED_IN]-임원
[269] | [법_제176조제4항_시세조종행위 등의 금지] | 매매-[INCLUDED_IN]-매매등
[319] | [법_제230조제1항_환매금지형집합투자기구] | 집합투자업자-[INCLUDED_IN]-집합투자업자등
[330] | [법_제235조제2항_환매청구 및 방법 등] | 해산-[INCLUDED_IN]-해산등
[361] | [법_제249조의18제2항_상호출자제한기업집단 계열 기관전용 사모집합투자기구 등에 대한 제한] | 기관전용 사모집합투자기구등-[SAME_AS]-기관전용 사모집합투자기구
[381] | [

In [11]:
for idx, triplet in enumerate(new_triplets):
    s, edge, o = triplet
    print(f"[{idx}] | {edge['reference']} : {s['keyword_name']}-[{edge['type']}]-{o['keyword_name']}")

[0] | 법_제3조제1항_금융투자상품 : 금전, 그 밖의 재산적 가치가 있는 것-[SAME_AS]-금전등
[1] | 법_제3조제1항_금융투자상품 : 위험-[SAME_AS]-투자성
[2] | 법_제3조제1항_금융투자상품 : 수익증권발행신탁이 아닌 신탁-[INCLUDED_IN]-관리형신탁
[3] | 법_제3조제1항_금융투자상품 : 수익권-[INCLUDED_IN]-관리형신탁
[4] | 법_제6조제7항_금융투자업 : 대통령령으로 정하는 투자대상자산-[INCLUDED_IN]-금융투자상품등
[5] | 법_제6조제7항_금융투자업 : 금융투자상품등에 대한 투자판단-[INCLUDED_IN]-투자자문업
[6] | 법_제8조제9항_금융투자업자 : 은행법 제2조의 은행-[SAME_AS]-은행
[7] | 법_제8조제9항_금융투자업자 : 보험업법 제2조의 보험회사-[SAME_AS]-보험회사
[8] | 법_제8조의2제5항_금융투자상품시장 등 : 매매체결대상상품-[INCLUDED_IN]-주권 및 그 밖에 대통령령으로 정하는 증권
[9] | 법_제8조의2제5항_금융투자상품시장 등 : 다자간매매체결업무-[INCLUDED_IN]-매매 또는 그 중개ㆍ주선이나 대리 업무
[10] | 법_제9조제10항_그 밖의 용어의 정의 : 상장법인-[SAME_AS]-증권시장에 상장된 증권을 발행한 법인
[11] | 법_제9조제10항_그 밖의 용어의 정의 : 상장증권-[INCLUDED_IN]-증권시장에 상장된 증권
[12] | 법_제9조제10항_그 밖의 용어의 정의 : 비상장법인-[SAME_AS]-상장법인을 제외한 법인
[13] | 법_제9조제10항_그 밖의 용어의 정의 : 한국금융투자협회-[SAME_AS]-협회
[14] | 법_제9조제10항_그 밖의 용어의 정의 : 한국예탁결제원-[SAME_AS]-예탁결제원
[15] | 법_제9조제10항_그 밖의 용어의 정의 : 제323조의3에 따라 인가를 받은 자-[SAME_AS]-금융투자상품거래청산회사
[16] | 법_제9조제10항_그 밖의 용어의 정의 : 제324조제1항

### enforcement_main

In [ ]:
import GraphDB.utils as utils

triplets = utils.get_triplet(result_path="../results/1-2/jsonl/01_enforcement_main_triplet_preprocessed_chunking_cot.jsonl", 
                             meta_path="../results/1-2/jsonl/01_enforcement_main_definition_metadata_preprocessed_chunking_cot.csv")
triplets

In [ ]:
len(triplets)

In [ ]:
for triplet in triplets:
    s, edge, o = triplet
    print(f"{edge['reference']} : {s['keyword_name']}-[{edge['type']}]-{o['keyword_name']}")


In [15]:
# 마음에 안드는 triplet idx 뽑아서 넣기
manual_remove_idx = [x for x in range(740, 767)]
remove_idx, new_triplets = process.postprocess(triplets, manual_remove_idx)
remove_idx.sort()

In [ ]:
print(f"original triplets : {len(triplets)}, new triplets : {len(new_triplets)}")

In [ ]:
for idx in remove_idx:
    s, edge, o = triplets[idx]
    print(f"[{idx}] | [{edge['reference']}] | {s['keyword_name']}-[{edge['type']}]-{o['keyword_name']}")

In [ ]:
for idx, triplet in enumerate(new_triplets):
    s, edge, o = triplet
    print(f"[{idx}] | {edge['reference']} : {s['keyword_name']}-[{edge['type']}]-{o['keyword_name']}")

### order_main

In [ ]:

import GraphDB.utils as utils
import GraphDB.process as process

triplets = utils.get_triplet(result_path="../results/1-2/jsonl/01_order_main_triplet_preprocessed_chunking_cot.jsonl", 
                             meta_path="../results/1-2/jsonl/01_order_main_definition_metadata_preprocessed_chunking_cot.csv")
len(triplets)

In [ ]:
for triplet in triplets:
    s, edge, o = triplet
    print(f"{edge['reference']} : {s['keyword_name']}-[{edge['type']}]-{o['keyword_name']}")

In [21]:
# 마음에 안드는 triplet idx 뽑아서 넣기
manual_remove_idx = []
remove_idx, new_triplets = process.postprocess(triplets, manual_remove_idx)
remove_idx.sort()

In [ ]:
print(f"original triplets : {len(triplets)}, new triplets : {len(new_triplets)}")

In [ ]:
for idx in remove_idx:
    s, edge, o = triplets[idx]
    print(f"[{idx}] | [{edge['reference']}] | {s['keyword_name']}-[{edge['type']}]-{o['keyword_name']}")


In [ ]:
for idx, triplet in enumerate(new_triplets):
    s, edge, o = triplet
    print(f"[{idx}] | {edge['reference']} : {s['keyword_name']}-[{edge['type']}]-{o['keyword_name']}")